<a href="https://colab.research.google.com/github/jcmachicao/MachineLearningAvanzado_UC_2024/blob/main/U2_MLADV__Torch_HYPER_Optuna_JCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimización de Hiperparametros de Red Neuronal OPTUNA

In [ ]:
!pip install optuna

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import optuna

In [3]:
# Define a simple feedforward neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [4]:
# Define the training function
def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(data.size(0), -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

In [5]:
# Define the evaluation function
def evaluate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data.view(data.size(0), -1))
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    accuracy = correct / len(val_loader.dataset)
    return val_loss, accuracy

In [6]:
# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to be optimized
    input_size = 28 * 28  # MNIST image size
    hidden_size = trial.suggest_int('hidden_size', 16, 256, log=True)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Create the model
    model = SimpleNN(input_size, hidden_size, 10)  # 10 classes for MNIST

    # Define the optimizer and criterion
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(3):  # Adjust as needed
        train(model, train_loader, optimizer, criterion)

    # Evaluate on validation set
    val_loss, accuracy = evaluate(model, val_loader, criterion)

    return val_loss

In [7]:
# Load MNIST data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, download=True, transform=transform), batch_size=64, shuffle=False)

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)  # You can adjust the number of trials

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print('Hidden Size: {}'.format(trial.params['hidden_size']))
print('Learning Rate: {}'.format(trial.params['learning_rate']))

100%|██████████| 9912422/9912422 [00:00<00:00, 169151353.77it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 45166179.65it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 48092482.80it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 12582912.00it/s]
[I 2024-02-21 05:19:06,312] A new study created in memory with name: no-name-44d85b74-159c-4adc-a6fe-e9f2e7ee01d1
<ipython-input-6-e2bb7f5abb25>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



[I 2024-02-21 05:19:39,359] Trial 0 finished with value: 0.004166220931429416 and parameters: {'hidden_size': 24, 'learning_rate': 0.0010219094468205837}. Best is trial 0 with value: 0.004166220931429416.
[I 2024-02-21 05:20:14,515] Trial 1 finished with value: 0.0034241532592102883 and parameters: {'hidden_size': 194, 'learning_rate': 0.00016157577115584877}. Best is trial 1 with value: 0.0034241532592102883.
[I 2024-02-21 05:20:47,937] Trial 2 finished with value: 0.005557313093915581 and parameters: {'hidden_size': 80, 'learning_rate': 4.719936207935048e-05}. Best is trial 1 with value: 0.0034241532592102883.
[I 2024-02-21 05:21:21,036] Trial 3 finished with value: 0.00510447137132287 and parameters: {'hidden_size': 45, 'learning_rate': 9.08371752212812e-05}. Best is trial 1 with value: 0.0034241532592102883.
[I 2024-02-21 05:21:56,974] Trial 4 finished with value: 0.0023232319671195 and parameters: {'hidden_size': 233, 'learning_rate': 0.00039120887014885004}. Best is trial 4 with 

Best trial:
Hidden Size: 233
Learning Rate: 0.00039120887014885004


Optuna Intro https://www.youtube.com/watch?v=t-INgABWULw